In [ ]:
import pandas as pd
import numpy as np

xlsx = pd.ExcelFile("Data5_23_Correct.xlsx", engine = "openpyxl")
dataOEE_Page = pd.read_excel(xlsx,'Twin Screw OEE')
dataTS_Page = pd.read_excel(xlsx, "Twin Screw ")